# Getting Started with LangSmith & LangChain Tracing

### Prerequisites and Crucial Tips

-   **Enable Tracing:** Set the environment variable before running your code:

```python
import os
os.environ["LANGCHAIN_TRACING"] = "true"
os.environ["LANGCHAIN_API_KEY"] = "<your-langsmith-api-key>"
```
*Note:* In my case I'll use a .env file to store these variables.

-   **Choose Your Decorator:**
-   Use `@traceable` for instrumenting standalone functions.

-   Use `@chain` when building LangChain runnables (or when you want automatic callback integration).

-   **Background Flushing:**\
    For short scripts, make sure to call langchain.callbacks.tracers.wait_for_all_tracers() at the end so that your asynchronous trace logs are sent before exit.

In [1]:
from dotenv import load_dotenv
from env_utils import doublecheck_env

# Load environment variables from .env
load_dotenv()

# Check and print results
doublecheck_env(".env")  # check environmental variables

LANGSMITH_API_KEY=****3d99
LANGSMITH_PROJECT=****dive
LANGSMITH_OTEL_ENABLED=true
LANGSMITH_TRACING=true
LANGSMITH_ENDPOINT=****.com
GOOGLE_API_KEY=****zZa4


---

## Example 1: Basic Function Tracing with @traceable

In [2]:
from langsmith import traceable

@traceable(run_type="tool")
def compute_sum(a: int, b: int) -> int:
    return a + b

result = compute_sum(3, 4)
print("compute_sum result:", result)  # Expected: 7

compute_sum result: 7


#### **Execution of the basic function tracing in LangSmith:**
<img src="./assets/basic_function_tracing.png" align="left" width="1300">

## Example 2: Nested Traceable Functions

In [3]:
from langsmith import traceable

@traceable(name="InnerJob")
def job_x(x: int) -> int:
    return x * 2

@traceable(name="OuterJob", tags=["batch"])
def job_y(values: list) -> int:
    total = 0
    for v in values:
        total += job_x(v)  # Nested trace: InnerJob becomes a child of OuterJob
    return total

result = job_y([1, 2, 3])
print("job_y result:", result)  # Expected: (1*2 + 2*2 + 3*2) = 12

job_y result: 12


*Crucial Info:*\
Nested calls automatically link together via context; no manual parent passing is needed.

#### **Execution of the nested function in LangSmith:**
<img src="./assets/nested_traceable_functions.png" align="left" width="1300">

## Example 3: Converting a Function into a Runnable with @chain

In [4]:
from langchain_core.runnables import chain
from langchain_core.tracers.context import tracing_v2_enabled

@chain
def greet(data: dict) -> str:
    # data should contain the key "person"
    person = data["person"]
    # Simulate a greeting message (could call an LLM here)
    return f"Hello, {person}!"

# Now, `greet` is a Runnable with .invoke() available.
with tracing_v2_enabled(project_name="GreetingApp"):
    result = greet.invoke({"person": "Alice"})
    print("Greeting result:", result)

Greeting result: Hello, Alice!


*Key Point:* The `@chain` decorator wraps your function into a Runnable that integrates with LangChain's callback system for tracing.

#### **Execution of Runnable in LangSmith:**
<img src="./assets/runnable.png" align="left" width="1300">

**Observation:** In langchain, the `create_agent()` function works with runnables.

## Example 4: Combining LangChain and Custom Tracing

> Langchain composes traces together using chaining, while langsmith composes traces together using scoping (i.e. outer functions calling inner functions).\
> This is how they can work together.

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate  # [!code ++]
# from langchain import LLMChain, PromptTemplate  # [!code --]
from langchain_core.tracers.context import tracing_v2_enabled
from langsmith import traceable, Client
import os

# Getting project name from .env file
project_name = os.getenv("LANGSMITH_PROJECT")

def parse_answer(text) -> str:
    # If the answer has a 'content' attribute (typical for ChatMessage objects), use it.
    if hasattr(text, "content"):
        text = text.content
    return f"Parsed answer: {text.strip()!r}"

@traceable(run_type="chain", project_name=project_name)
def perform_chain():
    """
    this function is traceable which means every chain runnning inside it
    will be composed together with it
    """

    # Compose a chain using the | operator (v1 style ✅).
    prompt = PromptTemplate.from_template("Tell me a joke about coding")  # [!code ++]
    llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")
    chain = prompt | llm  # ✅ LCEL (LangChain Expression Language) - v1 standard
    
    input_data = {}  # No inputs needed for this simple prompt.
    answer = chain.invoke(input_data)
    final_answer = parse_answer(answer)
    return final_answer

# Trace the overall process by using the tracing context manager.
with tracing_v2_enabled(project_name=project_name):
    answer = perform_chain()
    print(answer)

Parsed answer: 'Why do programmers prefer dark mode?\n\nBecause light attracts bugs!'


> The composition of chains using `prompt | llm` was employed, which represents the modern LangChain Expression Language (LCEL) pattern in v1. This operator is the recommended method and entirely replaces legacy approaches such as `LLMChain`.

> By applying the `@traceable` decorator with `run_type='chain'` and `project_name` to the `perform_chain` function, it was ensured that everything within it is traced as sub-runs. This includes the `prompt | llm` chain and its invocation, as well as the internal call to `parse_answer`, which is automatically traced without the need for an explicit decorator. This validates that the decorator correctly captures all nested operations.

> The use of the `tracing_v2_enabled` context manager with its `project_name` established a root trace in the LangSmith project. All internal calls, such as the execution of `perform_chain`, automatically inherit this context, ensuring a cohesive tracing hierarchy. This behavior confirms the proper setup and inheritance of the tracing context.

> As a result, a well-defined trace hierarchy was obtained in LangSmith, where `perform_chain` acts as the main execution of type 'chain'. Within it, the `prompt | llm` chain is detailed with its `prompt` and `ChatGoogleGenerativeAI` components, and the `parse_answer` function appears as a sub-run. This validates that the tracing structure was configured clearly, technically, and precisely, ready for documentation.

#### **Execution of the combined LangChain and Custom Tracing in LangSmith:**
<img src="./assets/combined_langchain_and_custom_tracing.png" align="left" width="1300">